In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document

In [5]:
loader1 = TextLoader("./data/nlp-keywords.txt", encoding='utf-8')
loader2 = TextLoader("./data/book_document.txt", encoding='utf-8')

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

In [7]:
# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

In [8]:
# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [10]:
# DB 생성
db = FAISS.from_documents(
    documents=split_doc1+split_doc2, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [11]:
# 문서(Document)로부터 추가 (add_documents)

In [13]:
# page_content, metadata 지정
db.add_documents(
    [
        Document(
            page_content="안녕하세요! 도큐먼트를 새로 추가해 보겠습니다.",
            metadata={"source": "mydata.txt"},
        )
    ],
    ids=["new_doc1"],
)

['new_doc1']

In [14]:
# 추가된 데이터를 확인
db.similarity_search("안녕하세요", k=1)

[Document(metadata={'source': 'mydata.txt'}, page_content='안녕하세요! 도큐먼트를 새로 추가해 보겠습니다.')]

In [15]:
# 추가된 데이터를 확인
db.index_to_docstore_id

{0: '97ada07d-3afa-4422-a6cc-9ff5e54d6fde',
 1: '5e4058b0-8f71-4792-a39f-61b4317b065e',
 2: '5e4a3c9c-9265-40a3-acff-a5df39c03c25',
 3: '6768cce7-04f0-435c-8339-3bcb7f9d3d77',
 4: '72575428-5fbf-467c-b596-14b3e45d0170',
 5: 'c9f06d9b-4fff-49e2-9560-6f9f45e07c62',
 6: '44190efd-6bf0-40d3-bf7e-213d1137370a',
 7: '21c88669-21d1-4b00-a511-3649e7373860',
 8: 'c240c7a5-226d-4610-83e0-9036a373a4aa',
 9: '3522d64c-a8c4-456f-823c-2f925fea8e76',
 10: 'dca5065c-d442-4536-ad68-3b52fa0ff0d7',
 11: 'b7889b2f-d9b3-47f1-91a9-bd90b2336167',
 12: '99ac9e63-6229-4afc-acd4-ba9a509b3efb',
 13: '0e0d61f5-4882-4a4b-9acc-42d9a16d523e',
 14: '421ed377-8a2f-4d89-952c-bc89af1009d6',
 15: 'new_doc1'}

In [17]:
# id 로 삭제
db.delete(['new_doc1'])

True

In [18]:
# 삭제된 결과를 출력
db.index_to_docstore_id

{0: '97ada07d-3afa-4422-a6cc-9ff5e54d6fde',
 1: '5e4058b0-8f71-4792-a39f-61b4317b065e',
 2: '5e4a3c9c-9265-40a3-acff-a5df39c03c25',
 3: '6768cce7-04f0-435c-8339-3bcb7f9d3d77',
 4: '72575428-5fbf-467c-b596-14b3e45d0170',
 5: 'c9f06d9b-4fff-49e2-9560-6f9f45e07c62',
 6: '44190efd-6bf0-40d3-bf7e-213d1137370a',
 7: '21c88669-21d1-4b00-a511-3649e7373860',
 8: 'c240c7a5-226d-4610-83e0-9036a373a4aa',
 9: '3522d64c-a8c4-456f-823c-2f925fea8e76',
 10: 'dca5065c-d442-4536-ad68-3b52fa0ff0d7',
 11: 'b7889b2f-d9b3-47f1-91a9-bd90b2336167',
 12: '99ac9e63-6229-4afc-acd4-ba9a509b3efb',
 13: '0e0d61f5-4882-4a4b-9acc-42d9a16d523e',
 14: '421ed377-8a2f-4d89-952c-bc89af1009d6'}

In [19]:
# 저장
db.save_local(folder_path="./faiss_db2", index_name="faiss_index2")

In [20]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./faiss_db2",
    index_name="faiss_index2",
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True,
)

In [21]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: '97ada07d-3afa-4422-a6cc-9ff5e54d6fde',
 1: '5e4058b0-8f71-4792-a39f-61b4317b065e',
 2: '5e4a3c9c-9265-40a3-acff-a5df39c03c25',
 3: '6768cce7-04f0-435c-8339-3bcb7f9d3d77',
 4: '72575428-5fbf-467c-b596-14b3e45d0170',
 5: 'c9f06d9b-4fff-49e2-9560-6f9f45e07c62',
 6: '44190efd-6bf0-40d3-bf7e-213d1137370a',
 7: '21c88669-21d1-4b00-a511-3649e7373860',
 8: 'c240c7a5-226d-4610-83e0-9036a373a4aa',
 9: '3522d64c-a8c4-456f-823c-2f925fea8e76',
 10: 'dca5065c-d442-4536-ad68-3b52fa0ff0d7',
 11: 'b7889b2f-d9b3-47f1-91a9-bd90b2336167',
 12: '99ac9e63-6229-4afc-acd4-ba9a509b3efb',
 13: '0e0d61f5-4882-4a4b-9acc-42d9a16d523e',
 14: '421ed377-8a2f-4d89-952c-bc89af1009d6'}

In [22]:
# 검색기로 변환
retriever = loaded_db.as_retriever()

In [23]:
retriever.invoke('파이썬 책이 있나요?')

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 손에잡히는 파이썬\n- 저자 : 문용준\n- 발행처 : BJpublic(비제이퍼블릭) 2018\n- isbn : 9791186697726\n- 청구기호 : 일 005.133-문66ㅅ\n- 등록번호 : CLP000178588\n- 소장위치 : 세 번째 책장 두 번 째 열\n\n\n### 제목 : 작심 3일 파이썬 Python\n- 저자 : 황덕창\n- 발행처 : 스포트라잇북

In [24]:
# MMR 검색 수행
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

In [26]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍

In [27]:
# MMR 알고리즘을 위해 더 많은 문서를 가져오되 상위 2개만 반환
# MMR 검색 수행, 상위 2개만 반환
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10})

In [30]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍

In [32]:
# 가장 유사한 단일 문서만 검색
# k=1 로 설정하여 가장 유사한 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})

In [33]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열')]